In [211]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import random
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report


In [212]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [213]:
# 1. Enhanced Data Loading & Preparation
# ======================

class PoseDataset(Dataset):
    def __init__(self, features, labels):
        # Normalize features
        self.features = torch.FloatTensor((features - features.mean(axis=0)) / (features.std(axis=0) + 1e-8))
        self.labels = torch.LongTensor(labels)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

def load_client_data(client_id, data_dir):
    """Load and preprocess client data"""
    features_path = os.path.join(data_dir, f'client_{client_id}_features.csv')
    labels_path = os.path.join(data_dir, f'client_{client_id}_labels.csv')
    
    features = pd.read_csv(features_path, header=None).values
    labels = pd.read_csv(labels_path, header=None).values.flatten()

    # Filter invalid labels and normalize
    valid_idx = (labels >= 0) & (labels < 12)
    features = features[valid_idx]
    labels = labels[valid_idx]
    
    return features, labels

def load_test_data(data_dir):
    """Load and preprocess test data"""
    test_features_path = os.path.join(data_dir, 'test_features.csv')
    test_labels_path = os.path.join(data_dir, 'test_labels.csv')
    
    test_features = pd.read_csv(test_features_path, header=None).values
    test_labels = pd.read_csv(test_labels_path, header=None).values.flatten()
    
    # Normalize test data using training stats (would normally use saved stats)
    test_features = (test_features - test_features.mean(axis=0)) / (test_features.std(axis=0) + 1e-8)
    
    return test_features, test_labels


In [214]:
# 2. Enhanced Model Architecture
# ======================

class EnhancedPoseClassifier(nn.Module):
    def __init__(self, input_size=270, num_classes=12):
        super(EnhancedPoseClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        return self.model(x)

In [215]:
# 3. Improved Federated Learning Setup
# ======================

def train_client(model, client_id, data_dir, epochs=5, batch_size=64):
    """Enhanced client training with learning rate scheduling"""
    features, labels = load_client_data(client_id, data_dir)
    dataset = PoseDataset(features, labels)
    
    # Split into train and validation (80/20)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1)
    
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_features, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_features, batch_labels in val_loader:
                outputs = model(batch_features)
                val_loss += criterion(outputs, batch_labels).item()
        model.train()
        
        scheduler.step(val_loss)
    
    return model.state_dict()

def aggregate_weights(client_weights, client_data_sizes):
    """Improved aggregation with momentum"""
    total_samples = sum(client_data_sizes)
    aggregated_weights = {}
    
    # Initialize with zeros as float tensors (fix type mismatch)
    for key in client_weights[0].keys():
        aggregated_weights[key] = torch.zeros_like(client_weights[0][key], dtype=torch.float)
    
    # Weighted average
    for idx, weights in enumerate(client_weights):
        weight_factor = client_data_sizes[idx] / total_samples
        for key in weights.keys():
            aggregated_weights[key] += weights[key] * weight_factor
    
    return aggregated_weights


def evaluate_model(model, test_features, test_labels):
    """Enhanced evaluation with full classification report"""
    model.eval()
    with torch.no_grad():
        test_tensor = torch.FloatTensor(test_features)
        outputs = model(test_tensor)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = accuracy_score(test_labels, predicted.numpy())
        
        # Print classification report
        print("\nClassification Report:")
        print(classification_report(test_labels, predicted.numpy(), zero_division=0))
        
    return accuracy


In [216]:
# 4. Enhanced FL Training Loop
# ======================

def federated_learning(data_dir, num_clients=10, rounds=30, clients_per_round=7, local_epochs=5, test_data_dir=None):
    if test_data_dir is None:
        test_data_dir = data_dir

    # Initialize global model
    global_model = EnhancedPoseClassifier(num_classes=12)
    
    # Load test data
    test_features, test_labels = load_test_data(test_data_dir)
    
    # Get client data sizes
    client_data_sizes = []
    for i in range(1, num_clients+1):
        _, labels = load_client_data(i, data_dir)
        client_data_sizes.append(len(labels))
    
    print("Client data distribution:", client_data_sizes)
    
    # Training tracking
    accuracies = []
    best_accuracy = 0
    
    # FL training loop
    for round in tqdm(range(rounds), desc="FL Training"):
        # Select clients - prioritize those with more data
        selection_probs = np.array(client_data_sizes) / sum(client_data_sizes)
        selected_clients = np.random.choice(num_clients, clients_per_round, p=selection_probs, replace=False)
        
        # Train on selected clients
        client_weights = []
        for client_id in selected_clients:
            local_model = EnhancedPoseClassifier(num_classes=12)
            local_model.load_state_dict(global_model.state_dict())
            
            updated_weights = train_client(local_model, client_id+1, data_dir, epochs=local_epochs)
            client_weights.append(updated_weights)
        
        # Aggregate updates
        selected_data_sizes = [client_data_sizes[client_id] for client_id in selected_clients]
        aggregated_weights = aggregate_weights(client_weights, selected_data_sizes)
        
        # Update global model with momentum
        current_state = global_model.state_dict()
        for key in current_state:
            current_state[key] = 0.9 * current_state[key] + 0.1 * aggregated_weights[key]
        global_model.load_state_dict(current_state)
        
        # Evaluate
        accuracy = evaluate_model(global_model, test_features, test_labels)
        accuracies.append(accuracy)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(global_model.state_dict(), 'best_global_model.pth')
        
        print(f"Round {round+1}: Test Accuracy = {accuracy:.4f} (Best: {best_accuracy:.4f})")
    
    # Plot training progress
    plt.figure(figsize=(10, 5))
    plt.plot(accuracies, label='Test Accuracy')
    plt.xlabel('FL Rounds')
    plt.ylabel('Accuracy')
    plt.title('Federated Learning Progress')
    plt.legend()
    plt.savefig('fl_training_progress.png')
    plt.close()
    
    return global_model


In [210]:
# 5. Run the Enhanced FL Process
# ======================

if __name__ == "__main__":
    # Set directories
    CLIENT_DATA_DIR = './client_datasets'
    TEST_DATA_DIR = '.'  # test files in main folder

    # Verify files
    for i in range(1, 11):
        assert os.path.exists(os.path.join(CLIENT_DATA_DIR, f'client_{i}_features.csv')), f"Missing client_{i}_features.csv"
        assert os.path.exists(os.path.join(CLIENT_DATA_DIR, f'client_{i}_labels.csv')), f"Missing client_{i}_labels.csv"
    
    assert os.path.exists(os.path.join(TEST_DATA_DIR, 'test_features.csv')), "Missing test_features.csv"
    assert os.path.exists(os.path.join(TEST_DATA_DIR, 'test_labels.csv')), "Missing test_labels.csv"

    print("All files verified. Starting Enhanced Federated Learning...")

    # Run FL with improved parameters
    final_model = federated_learning(
        data_dir=CLIENT_DATA_DIR,
        test_data_dir=TEST_DATA_DIR,
        num_clients=10,
        rounds=30,  # Increased rounds
        clients_per_round=7,  # More clients per round
        local_epochs=5  # More local epochs
    )

    # Final evaluation
    test_features, test_labels = load_test_data(TEST_DATA_DIR)
    final_accuracy = evaluate_model(final_model, test_features, test_labels)
    print(f"\nFinal Model Test Accuracy: {final_accuracy:.4f}")

All files verified. Starting Enhanced Federated Learning...
Client data distribution: [314, 113, 365, 207, 209, 119, 388, 142, 382, 64]


FL Training:   3%|▎         | 1/30 [00:00<00:28,  1.01it/s]


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00        57
           2       0.00      0.00      0.00        51
           3       0.00      0.00      0.00        56
           4       0.00      0.00      0.00        47
           5       0.12      0.60      0.20        50
           6       0.00      0.00      0.00        47
           7       0.00      0.00      0.00        37
           8       0.00      0.00      0.00        17
           9       0.00      0.00      0.00        12
          10       0.00      0.00      0.00        63
          11       0.12      0.56      0.20        52
          12       0.00      0.00      0.00        11

    accuracy                           0.12       500
   macro avg       0.02      0.09      0.03       500
weighted avg       0.02      0.12      0.04       500

Round 1: Test Accuracy = 0.1180 (Best: 0.1180)


FL Training:   7%|▋         | 2/30 [00:01<00:27,  1.01it/s]


Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        57
           2       0.00      0.00      0.00        51
           3       0.00      0.00      0.00        56
           4       0.00      0.00      0.00        47
           5       0.12      0.70      0.21        50
           6       0.00      0.00      0.00        47
           7       0.00      0.00      0.00        37
           8       0.00      0.00      0.00        17
           9       0.00      0.00      0.00        12
          10       0.00      0.00      0.00        63
          11       0.12      0.46      0.19        52
          12       0.00      0.00      0.00        11

    accuracy                           0.12       500
   macro avg       0.02      0.10      0.03       500
weighted avg       0.02      0.12      0.04       500

Round 2: Test Accuracy = 0.1180 (Best: 0.1180)


...

FL Training:  97%|█████████▋| 29/30 [00:30<00:01,  1.13s/it]


Classification Report:
              precision    recall  f1-score   support

           1       0.57      0.21      0.31        57
           2       0.50      0.02      0.04        51
           3       0.00      0.00      0.00        56
           4       0.38      0.17      0.24        47
           5       0.38      0.80      0.52        50
           6       0.29      0.51      0.37        47
           7       0.00      0.00      0.00        37
           8       0.07      0.41      0.12        17
           9       0.11      0.83      0.19        12
          10       0.62      0.08      0.14        63
          11       0.40      0.54      0.46        52
          12       0.00      0.00      0.00        11

    accuracy                           0.27       500
   macro avg       0.28      0.30      0.20       500
weighted avg       0.34      0.27      0.22       500

Round 29: Test Accuracy = 0.2700 (Best: 0.2700)


FL Training: 100%|██████████| 30/30 [00:31<00:00,  1.05s/it]


Classification Report:
              precision    recall  f1-score   support

           1       0.57      0.21      0.31        57
           2       0.50      0.02      0.04        51
           3       0.00      0.00      0.00        56
           4       0.40      0.17      0.24        47
           5       0.40      0.78      0.53        50
           6       0.28      0.53      0.37        47
           7       0.00      0.00      0.00        37
           8       0.07      0.41      0.12        17
           9       0.11      0.83      0.19        12
          10       0.67      0.10      0.17        63
          11       0.41      0.56      0.47        52
          12       0.00      0.00      0.00        11

    accuracy                           0.27       500
   macro avg       0.28      0.30      0.20       500
weighted avg       0.35      0.27      0.23       500

Round 30: Test Accuracy = 0.2740 (Best: 0.2740)



Classification Report:
              precision    recall  f1-score   support

           1       0.57      0.21      0.31        57
           2       0.50      0.02      0.04        51
           3       0.00      0.00      0.00        56
           4       0.40      0.17      0.24        47
           5       0.40      0.78      0.53        50
           6       0.28      0.53      0.37        47
           7       0.00      0.00      0.00        37
           8       0.07      0.41      0.12        17
           9       0.11      0.83      0.19        12
          10       0.67      0.10      0.17        63
          11       0.41      0.56      0.47        52
          12       0.00      0.00      0.00        11

    accuracy                           0.27       500
   macro avg       0.28      0.30      0.20       500
weighted avg       0.35      0.27      0.23       500


Final Model Test Accuracy: 0.2740
